#3.YAPAY SİNİR AĞI TASARLAYARAK MODELİN PERFORMANSINI DEĞERLENDİRME VE TEST VERİSİNİN ACCURACY, PRECISION, RECALL VE F1 SKORU, KARMAŞIKLIK MATRİSİ, CLASSIFICATION REPORT'UNU GÖSTERME

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

In [6]:
print(os.listdir("/gdrive/MyDrive/Kidney_Cancer/Kidney Cancer"))

DATADIR = '/gdrive/MyDrive/Kidney_Cancer/Kidney Cancer'

['Tumor', 'Normal']


In [9]:
CATEGORIES = ["Tumor", "Normal"]

#Görüntü pikseli boyutlandırma
IMG_SIZE = 100

In [10]:
#Veri Setini İşleme ve Liste Oluşturma

all_data = []
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img))
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            all_data.append([new_array, class_num])
        except Exception as e:
            pass


In [11]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  10000


In [12]:
# Veri Setini Özellik ve Etiket olarak NumPy Dizilerine Ayırma
X = []
y = []

for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)


In [13]:
# Veriyi normalize etme
X = X / 255.0

In [14]:
# Veri Setini Eğitim, Test ve Doğrulama Olarak Bölme
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state =42)
# Eğitim verisini bölerek doğrulama seti oluşturma
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [15]:
#Eğitim, Test ve Doğrulama Veri Setlerini NumPy Dizilerine Dönüştürme
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [16]:
#Veri Seti Boyutlarını Kontrol Etme
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(5850, 100, 100, 3)
(3500, 100, 100, 3)
(650, 100, 100, 3)
(5850,)
(3500,)
(650,)


In [18]:
# Etiketleri One-Hot Encoding Formatına Çevirme
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)



In [19]:
# Yapay Sinir Ağı Modeli
model_nn = Sequential()
model_nn.add(Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model_nn.add(Dense(256, activation='relu'))
model_nn.add(Dropout(0.4))
model_nn.add(Dense(128, activation='relu'))
model_nn.add(Dropout(0.4))
model_nn.add(Dense(2, activation='softmax'))

In [20]:
# Modeli derleme
model_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
#Geri Çağrılar Oluşturma
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [22]:
# Modeli eğitme
history = model_nn.fit(x_train, train_yCl,
                    batch_size=64,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 30
                    )

Epoch 1/30
92/92 [==============================] - ETA: 0s - loss: 0.5015 - accuracy: 0.8174
Epoch 1: val_accuracy improved from -inf to 0.97231, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


92/92 [==============================] - 22s 223ms/step - loss: 0.5015 - accuracy: 0.8174 - val_loss: 0.1016 - val_accuracy: 0.9723
Epoch 2/30
92/92 [==============================] - ETA: 0s - loss: 0.1096 - accuracy: 0.9653
Epoch 2: val_accuracy improved from 0.97231 to 0.98615, saving model to model.h5
92/92 [==============================] - 15s 159ms/step - loss: 0.1096 - accuracy: 0.9653 - val_loss: 0.0391 - val_accuracy: 0.9862
Epoch 3/30
92/92 [==============================] - ETA: 0s - loss: 0.0975 - accuracy: 0.9648
Epoch 3: val_accuracy improved from 0.98615 to 0.98769, saving model to model.h5
92/92 [==============================] - 14s 157ms/step - loss: 0.0975 - accuracy: 0.9648 - val_loss: 0.0392 - val_accuracy: 0.9877
Epoch 4/30
92/92 [==============================] - ETA: 0s - loss: 0.0812 - accuracy: 0.9716
Epoch 4: val_accuracy did not improve from 0.98769
92/92 [==============================] - 12s 131ms/step - loss: 0.0812 - accuracy: 0.9716 - val_loss: 0.0379 

In [23]:
# Modelin performansını değerlendirme
score_nn_val_transfer = model_nn.evaluate(x_val, valid_yCl)
print("Validation Accuracy (Neural Network): {:.2f}%".format(score_nn_val_transfer[1] * 100))

score_nn_test_transfer = model_nn.evaluate(x_test, test_yCl)
print("Test Accuracy (Neural Network): {:.2f}%".format(score_nn_test_transfer[1] * 100))

score_nn_train_transfer = model_nn.evaluate(x_train, train_yCl)
print("Train Accuracy (Neural Network): {:.2f}%".format(score_nn_train_transfer[1]))


21/21 [==============================] - 0s 17ms/step - loss: 0.0221 - accuracy: 0.9923
Validation Accuracy (Neural Network): 99.23%
110/110 [==============================] - 2s 18ms/step - loss: 0.0249 - accuracy: 0.9911
Test Accuracy (Neural Network): 99.11%
183/183 [==============================] - 6s 32ms/step - loss: 0.0049 - accuracy: 0.9983
Train Accuracy (Neural Network): 1.00%


In [24]:
# Tahmin yapma
y_pred_nn = model_nn.predict(x_test)
y_pred_classes_nn = np.argmax(y_pred_nn, axis=1)


110/110 [==============================] - 3s 22ms/step


In [26]:

# Accuracy, Precision, Recall, F1 Score
accuracy_nn = accuracy_score(y_test, y_pred_classes_nn)
precision_nn = precision_score(y_test, y_pred_classes_nn)
recall_nn = recall_score(y_test, y_pred_classes_nn)
f1_nn = f1_score(y_test, y_pred_classes_nn)

print("Neural Network Test Accuracy: {:.2f}".format(accuracy_nn))
print("Neural Network Test Precision: {:.2f}".format(precision_nn))
print("Neural Network Test Recall: {:.2f}".format(recall_nn))
print("Neural Network Test F1 Score: {:.2f}".format(f1_nn))

# Karmaşıklık Matrisi
conf_matrix_nn = confusion_matrix(y_test, y_pred_classes_nn)
print("Neural Network Confusion Matrix:\n", conf_matrix_nn)

# Classification Report
class_report_nn = classification_report(y_test, y_pred_classes_nn)
print("Neural Network Classification Report:\n", class_report_nn)


Neural Network Test Accuracy: 0.99
Neural Network Test Precision: 0.99
Neural Network Test Recall: 0.99
Neural Network Test F1 Score: 0.99
Neural Network Confusion Matrix:
 [[1765   21]
 [  10 1704]]
Neural Network Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      1786
           1       0.99      0.99      0.99      1714

    accuracy                           0.99      3500
   macro avg       0.99      0.99      0.99      3500
weighted avg       0.99      0.99      0.99      3500

